# Module 4, Activity 1: Building a Simple AI Agent

Now we are going to start tying everything together in the creation of an AI agent.  To do so, you will see in this module how to create tools and add them to the LLM to form agents.

In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached cffi-1.17.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.8 MB/s eta 0:00:00a 0:00:01
Using cached cffi-1.17.1-cp312-cp312-macosx_11_0_arm64.whl (178 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Created wheel for peewee: filename=peewee-3.18.1-cp312-cp312-macosx_11_0_arm64.whl size=272130 sha256=34bee26985d92347dd92adf54025249854d329ea1861456914dcb0b5323d7d35
  Stored in directory: /Users/skkakollu/Library/Caches/pip/wheels/1a/57/6a/bb71346381d0d911cd4ce3026f1fa720da76707e4f01cf27dd
Successfully built peewee

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upg

In [3]:
import boto3
import csv
from datetime import datetime, timedelta
import io
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import yfinance as yf


from langchain.agents import AgentType, AgentExecutor, create_tool_calling_agent, initialize_agent
from langchain_aws import ChatBedrockConverse 
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import tool

In [4]:
session = boto3.session.Session()
region = session.region_name
bedrock_runtime = boto3.client("bedrock-runtime", region_name='us-west-2')

## Creating Tools

A tool is a function that you create that augments the capabilities of the LLM.  Tools can be anything, including calls to APIs.  In the case below, we are going to use the common `yfinance` Python package to create a simple tool that downloads data from the Yahoo! Finance API.  We specify our tools with the LangChain `@tool` decorator, which takes a normal Python function and turns it into a LangChain tool that can be used by an agent.

**Important Note:** It is important to check out the tool description in the docstring of the function.  LangChain will use these tool descriptions as a natural language way to understand what each tool does and then route relevant work to that tool.  So be sure to be specific in your descriptions!

In [5]:
@tool
def get_stock_history(input: str) -> str:
    """
    Get historical stock data for a ticker symbol.
    Input format: 'TICKER, PERIOD, INTERVAL'. Example: 'BILL, 10d, 1d'.
    Period is optional (default '5d'). Interval is optional (default '1d').
    """
    try:
        parts = [p.strip() for p in input.split(",")]
        ticker = parts[0]
        period = parts[1] if len(parts) > 1 else "5d"
        interval = parts[2] if len(parts) > 2 else "1d"
        data = yf.download(ticker, period=period, interval=interval)
        if data.empty:
            return f"No data found for {ticker}."
        return data.to_string()
    except Exception as e:
        return f"Error fetching data: {str(e)}"


Agents can use any number of tools.  We will collect them into a list that then gets passed to the agent.

In [ ]:
tools = [get_stock_history]

In [6]:
llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0.0,
)

## Initializing the basic agent

We now have our tools and LLM so it is time to create an agent with them.  We note that there are many different types of basic agent types that LangChain can use.  Descriptions can be found [here](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html#langchain.agents.agent_types.AgentType).  Be sure to try a few out and see how they vary.

In [7]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

NameError: name 'tools' is not defined

In [ ]:
response = agent.invoke("Show me the stock history for BILL over the past 10 days")
print(response)

## Adding more tools

Let's now create another tool that can plot our data.  So the idea would be that if the user asks to plot the stock price, the first tool to be called would be the `get_stock_history` tool we created above.  Then -- and only if the user requests the information be plotted -- the following tool gets used to create the plot.

**Special Note:** You will see that we have included this `|||` delimiter below.  Because the agent type we are using (`ZERO_SHOT_REACT_DESCRIPTION`) takes only a single string, we want to provide guidance to the LLM on how to separate the CSV data from the ticker.  Using a custom delimiter like `|||` is helpful because CSV data often contains commas, line breaks, and quotes — so simple split(',') parsing would break. Claude will usually handle this fine if you describe the format in the docstring.

In [ ]:
@tool
def plot_stock_data(input: str) -> str:
    """
    Create a line plot of the stock's closing price from CSV data.
    Input format: 'CSV_DATA ||| TICKER'. Example: '<csv text> ||| BILL'
    Returns the file path of the saved plot image.
    """
    try:
        # Separate CSV and ticker
        if "|||" in input:
            csv_data, ticker = input.split("|||", 1)
            ticker = ticker.strip()
        else:
            return "Input must be in the format: 'CSV_DATA ||| TICKER'"

        df = pd.read_csv(io.StringIO(csv_data.strip()))

        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
        else:
            return "No 'Date' column found."

        if 'Close' not in df.columns:
            return "No 'Close' column found to plot."

        plt.figure(figsize=(10, 5))
        plt.plot(df['Date'], df['Close'], marker='o', linestyle='-')
        plt.title(f"{ticker} Stock Price")
        plt.xlabel("Date")
        plt.ylabel("Closing Price")
        plt.grid(True)

        image_path = f"{ticker}_plot.png"
        plt.savefig(image_path)
        plt.close()
        return f"Plot saved as {image_path}"
    except Exception as e:
        return f"Error generating plot: {str(e)}"


## Agent Executors

When we start adding more tools in or the possibility of using multiple tools like this case, we need something to help with the orchestration of that work (as opposed to just directly passing the user's query into the LLM like we did above).  So we introduce the `AgentExecutor`, whose purpose is to act like an air traffic controller, sending the necessary information from one tool to another.

You will notice that the prompt has also gotten a bit more complicated.  Let's talk about a few of these different elements:

`{chat_history}`: This is the traditional memory like we had before.  It captures the exchanges between the user and the agent and is used to give context to the LLM.

`{agent_scratchpad}`: This stores the LLM's reasoning and intermediate tools calls.  It shows what the LLM is currently working on, like thoughts, tools, and results.  This is really important when you start using multiple tools.

**Note on memory:** You will likely see a deprecation warning about memory.  This is because LangChain is encouraging people to migrate to LangGraph for checkpointer-based memory.  As previously stated, LangGraph has a pretty steep learning curve and is beyond the scope of this workshop.  So it is OK to ignore the deprecation warning.

In [ ]:
tools = [get_stock_history, plot_stock_data]

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

## Agent types

`initialize_agent` is a convenience function that simplifies agent creation.  You will note that you need to pass to it an agent type.  There are many different agent types that you can use, which are documented [here](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.agent_types.AgentType.html).  Note that not all agents work with all models (ex: `OPENAI_FUNCTIONS`).  In this case, we are using a zero-shot ReAct (Reasoning + Acting) agent, which is a simple place to start.

In [ ]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)

response = agent_executor.invoke({"input": "Plot the last 15 days of BILL stock data."})
print(response["output"])

And hopefully you now have a `.png` file that was created in this directory of the stock price!  (You might need to reload the files in the panel on the left.)

## Concluding Thoughts

Agents are all the rage right now in GenAI.  As such, working with agents is a very hot area with a lot of development.  As you read through various bits of documentation on LangChain, you will no doubt come across LangGraph, which is LangChain's new approach to working with agents, including multi-agent workflows.  

LangGraph is very powerful and LangChain is moving all of its agentic workflows to this platform.  However, it is still quite new and in a constant state of flux.  Additionally, it has a very steep learning curve and the documentation has not caught up to its development yet.  Because of this, we have stayed away from it for this workshop since it would be very difficult to cover it within a single day's class.  That being said, the interested student should definitely take a look at it since this is where all of LangChain's work is moving towards in the future.